In [ ]:
%matplotlib inline
import numpy as np

In [ ]:
def compress_state(psi,L,d,chi_max, tol = 10**(-14)):
        A,s,V = np.linalg.svd(np.reshape(psi,[d,d**(L-1)]),full_matrices=0)

        A_list = []
        chi = np.min([np.sum(s>tol), chi_max])
        A_list.append(A[:,:chi])
        for i in range(1,L-1):
                psi = np.tensordot(np.diag(s),V,axes=(1,0))[:chi,:]

                A,s,V = np.linalg.svd(np.reshape(psi,[chi*d,d**(L-i-1)]),full_matrices=0)

                A = np.reshape(A,[chi,d,-1])
                chi = np.min([np.sum(s>tol), chi_max])
                A_list.append(np.transpose(A[:,:,:chi],(1,0,2)))

        A_list.append(np.transpose(np.tensordot(np.diag(s),V,axes=(1,0))[:chi,:],(1,0)))

        return A_list

def expand_state(A_list):
    L = len(A_list)
    d = A_list[0].shape[0]

    psi = A_list[0]
    for i in range(1,L):
        psi = np.tensordot(psi,A_list[i],axes=(i,1))
    return np.reshape(psi,d**L)

In [ ]:
L = 10
d = 2
chi_max = 16

psi_full = np.random.rand(d**L)
psi_full = psi_full / np.linalg.norm(psi_full)

A_list = compress_state(psi_full,L,d,chi_max)

psi_compressed = expand_state(A_list)
psi_compressed = psi_compressed / np.linalg.norm(psi_compressed)

print("Fidelity: ",np.inner(psi_full,psi_compressed))

Fidelity:  0.9859485263384302
